In [1]:
import os
import sys
module_path = "/data/luojaa/eukgen/"
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
#get default altair style'
%run ~/data/luojaa/eukgen/scripts/altair_style_config_default.py


In [2]:
# cluster size distribution
# distribution of euk/prok members

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from core_functions.altair_plots import plot_cumsum_counts

In [68]:
kogs_df

,UID,KOGID,FUNCTION,ALIAS,KEGG_ORG,ISEUK,TAXID,SPECIES
0,A0A022Q707,K00001,alcohol dehydrogenase [EC:1.1.1.1],none,EGT,True,4155,Erythranthe guttata
1,A0A022R7F5,K00001,alcohol dehydrogenase [EC:1.1.1.1],none,EGT,True,4155,Erythranthe guttata
2,A0A059A4S5,K00001,alcohol dehydrogenase [EC:1.1.1.1],none,EGR,True,71139,Eucalyptus grandis
3,A0A059A5P0,K00001,alcohol dehydrogenase [EC:1.1.1.1],none,EGR,True,71139,Eucalyptus grandis
4,A0A059ZDV9,K00001,alcohol dehydrogenase [EC:1.1.1.1],none,ABW,False,470,Acinetobacter baumannii
...,...,...,...,...,...,...,...,...
27895405,none,K27109,none,ctaF,TPUL,False,0,none
27895406,none,K27109,none,none,TSD,False,0,none
27895407,none,K27109,none,none,WHR,False,0,none
27895408,none,K27109,none,none,YIA,False,0,none


In [4]:
kogs_df_searchable = "/data/luojaa/uid_stats/kogs_df_searchable.tsv"
kogs_df = pd.read_csv(kogs_df_searchable, sep = "\t")
KOGS_w_both = "/data/luojaa/uid_stats/KOGs_w_both.csv"
relevant_kogs = pd.read_csv(KOGS_w_both)["KOG"]

accessions_df = kogs_df.loc[:,["KOGID", "UID", "ISEUK", "SPECIES"]]
accessions_df = accessions_df[accessions_df["SPECIES"] != "none"]
accessions_df = accessions_df[accessions_df["KOGID"].isin(relevant_kogs)]
accessions_df = accessions_df.set_index("UID")

In [5]:
taxids_path = "/data/luojaa/eukgen/mmseqs/taxids_classlabels.tsv"
taxids_classes_df = pd.read_csv(taxids_path, sep = "\t", header = None).rename(columns = {0:"UID", 1:"TAXID", 2:"SUPERKINGDOM", 3:"CLASS"}).set_index("UID")

In [6]:
cluster_data_df = pd.merge(accessions_df, taxids_classes_df,on = "UID", how = "left")

In [7]:
cluster_data_df = cluster_data_df.reset_index()


In [8]:
kog_original_sizes = kogs_df["KOGID"].value_counts()

In [9]:
microcosm_sizes = cluster_data_df["KOGID"].value_counts()

In [66]:
microcosm_sizes

KOGID
K07497    27096
K00059    20672
K00799    19709
K02003    16990
K00626    13846
          ...  
K22239       17
K23272       17
K04126       12
K12743       11
K23592        6
Name: count, Length: 2334, dtype: int64

In [57]:
plot, data1 = plot_cumsum_counts(microcosm_sizes, title='KEGG microcosm size distribution', plot_type='default')
plot, data2 = plot_cumsum_counts(kog_original_sizes, title='KEGG group size distribution', plot_type='default')
all_data = pd.concat([data1, data2])
plot, data = plot_cumsum_counts(all_data, formatted_data=True, title='KOG size distributions', plot_type='default', x_label = "Number of proteins in group")
plot

alt.LayerChart(...)

In [21]:
microcosm_sizes_df = pd.DataFrame(microcosm_sizes)

In [17]:
kog_iseuk = cluster_data_df.loc[:,["KOGID", "ISEUK"]].groupby("KOGID").agg(sum).reset_index().set_index("KOGID")

/tmp/ipykernel_3881369/787565939.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  kog_iseuk = cluster_data_df.loc[:,["KOGID", "ISEUK"]].groupby("KOGID").agg(sum).reset_index().set_index("KOGID")


In [22]:
kog_euk_stats = pd.merge(kog_iseuk, microcosm_sizes_df, on = "KOGID", how = "left")

In [24]:
kog_euk_stats["perc_euk"] = kog_euk_stats["ISEUK"] / kog_euk_stats["count"]

In [26]:
kog_perc_euk = kog_euk_stats["perc_euk"]

In [58]:

alt.Chart(kog_perc_euk.reset_index()).mark_bar().encode(
    alt.X("perc_euk:Q").bin().title("proportion of eukaryotic proteins in microcosm"),
    y=alt.Y('count()').title("number of microcosms"),
).properties(title='% euk proteins across microcosms')

alt.Chart(...)

In [69]:
np.mean(kog_perc_euk), np.median(microcosm_sizes), np.median(kog_original_sizes)

(0.4624139176159575, 823.0, 458.0)